## importing

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
from sklearn import svm, tree, preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_curve, auc, r2_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
lb = preprocessing.LabelBinarizer()
scaler = MinMaxScaler()
#classifier = LogisticRegression()
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
# import pandas as pd
# import numpy as np
# import pandas_profiling
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.decomposition import PCA
# from time import time
# from vpython import *
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# #import pprint
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# #from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV
# %matplotlib inline
# from sklearn import svm
# from sklearn.metrics import classification_report
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import tree
# from sklearn.ensemble import AdaBoostClassifier

In [3]:
def model_report(model): 
    train_score = model.score(X_train, y_train)
    print("train score: {:.4}%".format(train_score * 100))

    test_score = model.score(X_test, y_test)
    print("test score: {:.4}%".format(test_score * 100))

    cv_score = np.mean(cross_val_score(model, X, y, cv=3))
    print("cross val score: {:.4}%".format(cv_score * 100))

    false_positive_rate, true_positive_rate, thresholds = roc_curve(actuals, predictions)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    print("AUC  : {:.4}%".format(roc_auc * 100))

    print("\nClassification Report:")
    print(classification_report(actuals, predictions))
    
    print("\nConfusion Matrix: 1=Fatal, 0=Minor")
    display(pd.crosstab(actuals, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [4]:
def confusion_matrix():
    print("\nConfusion Matrix: 1=Fatal, 0=Minor")
    display(pd.crosstab(actuals, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True))

In [5]:
df = pd.read_csv('Accidents_categorical.csv')

## eda

In [6]:
#df.head()

In [7]:
#df.shape

In [8]:
#df.head()

In [9]:
#pandas_profiling.ProfileReport(df)

In [10]:
#df.plot(kind="scatter", x="Longitude", y="Latitude", alpha= 0.1)

In [11]:
#df.hist(figsize=(20,15), bins=30);

In [12]:
#df.info()

In [13]:
#pairplot and heatmap corr plot

## data prep

In [14]:
#taking 5% of data for speedier results before committing to model
df = df.sample(frac=.01, random_state=3)

In [15]:
df["Accident_Severity"]= df['Accident_Severity'].replace("Fatal_Serious", 1)
df["Accident_Severity"]= df['Accident_Severity'].replace("Slight", 0)

In [16]:
target = df['Accident_Severity']

In [17]:
# make hour of day
df['Hour_of_Day'] = df['Hour_of_Day'].apply(lambda x: round(x*24))
df['Hour_of_Day'].unique()

array([ 8, 23, 17, 24, 19, 12, 10, 18, 16, 14, 15, 22, 13, 20,  7, 11,  9,
        6,  2, 21,  5,  1,  0,  3,  4])

In [18]:
# assign target and drop irrelevant features
y = df["Accident_Severity"]
df.drop(columns=['Accident_Index','Datetime'
                 , 'Accident_Severity'
                 , 'Year'], axis=1,inplace=True)

In [19]:
#selecting categorical features for dummies
col_list = list(df.select_dtypes(include=['object']).columns)
for col in ('Season','Day_of_Month','Day_of_Week','Month_of_Year','Hour_of_Day'):
    col_list.append(col)

In [20]:
#col_list

In [21]:
df = pd.DataFrame(pd.get_dummies(data=df, columns=col_list, drop_first=True
                    , prefix=col_list))

In [22]:
X = df

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [24]:
# y_train_fatal = (y_train=="1")
# y_test_fatal = (y_test=="1")

In [25]:
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

## feature importance

In [26]:
# model = XGBClassifier()
# model.fit(X, y)
# ax = plot_importance(model)
# fig = ax.figure
# ax.grid(False)
# fig.set_size_inches(10,20)
# plt.show()

In [27]:
# from sklearn.ensemble import ExtraTreesClassifier
# model = ExtraTreesClassifier()
# model.fit(X,y)
# #plot graph of feature importances
# feat_importances = pd.Series(model.feature_importances_, index=X.columns)
# feat_importances.nlargest(20).plot(kind='barh')
# plt.show()

## choosing model

In [28]:
X_train.shape

(2014, 178)

In [29]:
# looking to keep enough to maintain 80% explained variance
pca = PCA(n_components=115)
principalComponents = pca.fit_transform(X_train)
print(np.sum(pca.explained_variance_ratio_))

0.8389852343601241


In [30]:
num_feats = 115

In [31]:
#assign pipelines
pipe_kn = Pipeline([('kn_model', KNeighborsClassifier())])

pipe_tree = Pipeline([('dt_model', DecisionTreeClassifier(random_state=3))])

pipe_bag = Pipeline([('bag_model', BaggingClassifier(DecisionTreeClassifier(random_state=3)))])

pipe_rf = Pipeline([('rf_model', RandomForestClassifier(random_state=3))])

pipe_ada = Pipeline([('ada_model', AdaBoostClassifier(random_state=3))])

pipe_gbt = Pipeline([('gbt_model', GradientBoostingClassifier(random_state=3))])

pipe_xgb = Pipeline([('xgb_model', XGBClassifier(random_state=3))])

pipe_svc = Pipeline([('svm_model', svm.SVC(random_state=3))])

In [32]:
# baseline model cross val scores
pipelines = [pipe_kn, pipe_tree, pipe_bag, pipe_rf, pipe_ada, pipe_gbt, pipe_xgb, pipe_svc]
pipeline_names = ['KNeighbors','Decision Tree', 'Bagged Trees', 'Random Forest', 'AdaBoost', 'Gradient Boosting', 'XGBoost', 'Logistic Regression', 'Support Vector']

for pipeline, name in zip(pipelines, pipeline_names):
    print(name, ':')
    print(pipeline)
    pipeline.fit(X_train, y_train)
    
    print()
    recall = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
    print('Recall', np.mean(recall))
    precision = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='precision')
    print('Precision', np.mean(precision))
    f1 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1')
    print('F1', np.mean(f1))
    print()
    print()

KNeighbors :
Pipeline(memory=None,
     steps=[('kn_model', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])

Recall 0.16275247524752476
Precision 0.3057390429999408
F1 0.2122676509249221


Decision Tree :
Pipeline(memory=None,
     steps=[('dt_model', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=3,
            splitter='best'))])

Recall 0.3255247524752475
Precision 0.32444039659978186
F1 0.3242087426239008


Bagged Trees :
Pipeline(memory=None,
     steps=[('bag_model', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_feature

While XGboost shows the highest precision, decision tree shows the highest recall rate and F1 score.

In [33]:
#baseline model accuracy scores

for classifier, pl in zip((XGBClassifier, LogisticRegression, RandomForestClassifier,DecisionTreeClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, svm.SVC),('xgbc','logistic regression', 'random forest', 'decision tree', 'adaboost', 'gradient', 'bagging', 'support vector')):

    pipe = Pipeline([('pca', PCA(n_components=num_feats)),
                    ('clf', classifier(random_state=3))])
    pipe.fit(X_train, y_train)
    
    print(pl)
    print(pipe.score(X_test, y_test))
    print()

xgbc
0.7420634920634921

logistic regression
0.7420634920634921

random forest
0.7242063492063492

decision tree
0.6111111111111112

adaboost
0.7123015873015873

gradient
0.7202380952380952

bagging
0.7043650793650794

support vector
0.7380952380952381



In [34]:
# for classifier, cls in zip((XGBClassifier(), LogisticRegression(), RandomForestClassifier(),DecisionTreeClassifier(), AdaBoostClassifier(),GradientBoostingClassifier(), BaggingClassifier(), svm.SVC() ),('xg','logistic regression', 'random forest', 'decision tree', 'adaboost', 'gradient', 'bagging', 'support vector')):
    
    
#     pipe_lr = Pipeline([('pca', PCA(n_components=150, random_state=3)),
#          ('clf', classifier(random_state=3))])

#     pipe_lr.fit(X_train, y_train)
    
# #     print(cls, ":")
# #     recall = cross_val_score(classifier, X_train, y_train, cv=5, scoring='recall')
# #     print('Recall', np.mean(recall))
# #     precision = cross_val_score(classifier, X_train, y_train, cv=5, scoring='precision')
# #     print('Precision', np.mean(precision))
# #     f1 = cross_val_score(classifier, X_train, y_train, cv=5, scoring='f1')
# #     print('F1', np.mean(f1))
# #     print()

## decision tree

In [35]:
# initialize and fit decision tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

predictions = dt_model.predict(X_test)
actuals = y_test

In [36]:
model_report(dt_model)

train score: 100.0%
test score: 64.68%
cross val score: 66.2%
AUC  : 55.24%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.75      0.76       370
           1       0.34      0.35      0.35       134

   micro avg       0.65      0.65      0.65       504
   macro avg       0.55      0.55      0.55       504
weighted avg       0.65      0.65      0.65       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,279,91,370
1,87,47,134
All,366,138,504


In [ ]:
dec_tree = model_report(dt_model)

In [37]:
# grid search params
param_grid_dt = [
    {'dt_model__criterion': ['entropy','gini'],
    'dt_model__max_depth': [3,4,5],
    'dt_model__min_samples_split': [0.01, 0.001],
    'dt_model__min_samples_leaf': [0.001, 0.01, .1]}
]

# grid search
gs_dt = GridSearchCV(estimator=pipe_tree,
                    param_grid=param_grid_dt,
                    scoring='accuracy',
                    cv=4, n_jobs=-1, verbose=1)

# fitting grid search
gs_dt.fit(X_train, y_train)

# best parameters
print('Best accuracy: {:.4}%'.format(gs_dt.best_score_ * 100))
print('params:\n', gs_dt.best_params_)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:    4.7s


Best accuracy: 75.67%
params:
 {'dt_model__criterion': 'entropy', 'dt_model__max_depth': 4, 'dt_model__min_samples_leaf': 0.01, 'dt_model__min_samples_split': 0.002}


[Parallel(n_jobs=-1)]: Done  89 out of  96 | elapsed:    5.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:    5.5s finished


In [38]:
# using best params to fit model
dt_gs_model = DecisionTreeClassifier(criterion='entropy',
                               max_depth=4,
                               min_samples_leaf=0.01,
                               min_samples_split=0.002)
dt_gs_model.fit(X_train, y_train)

predictions = dt_gs_model.predict(X_test)
actuals = y_test

In [39]:
# new model report
model_report(dt_gs_model)

train score: 77.06%
test score: 73.21%
cross val score: 75.14%
AUC  : 53.67%

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.95      0.84       370
           1       0.48      0.12      0.19       134

   micro avg       0.73      0.73      0.73       504
   macro avg       0.62      0.54      0.52       504
weighted avg       0.68      0.73      0.67       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,353,17,370
1,118,16,134
All,471,33,504


## Random Forest

In [40]:
#initialize and fit random forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
actuals = y_test

In [41]:
#baseline model report
model_report(rf_model)

train score: 97.27%
test score: 72.82%
cross val score: 74.11%
AUC  : 54.59%

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.94      0.83       370
           1       0.47      0.16      0.23       134

   micro avg       0.73      0.73      0.73       504
   macro avg       0.61      0.55      0.53       504
weighted avg       0.68      0.73      0.68       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,346,24,370
1,113,21,134
All,459,45,504


In [42]:
# grid search params
param_grid_rf = [
    {'rf_model__criterion': ['entropy','gini'],
    'rf_model__n_estimators': [100,150],
    'rf_model__max_depth': [None],
    'rf_model__min_samples_split': [0.01,0.001],
    'rf_model__min_samples_leaf': [0.01,0.001]}
]

# grid search
gs_rf = GridSearchCV(estimator=pipe_rf,
                    param_grid=param_grid_rf,
                    scoring='accuracy',
                    cv=4, n_jobs=-1, verbose=1)

# fitting grid search
gs_rf.fit(X_train, y_train)

# best parameters
print('best accuracy: {:.4}%'.format(gs_rf.best_score_ * 100))
print('params:\n', gs_rf.best_params_)

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   18.9s finished


best accuracy: 76.12%
params:
 {'rf_model__criterion': 'gini', 'rf_model__max_depth': None, 'rf_model__min_samples_leaf': 0.001, 'rf_model__min_samples_split': 0.001, 'rf_model__n_estimators': 100}


In [43]:
# fitting model using best params
rf_gs_model = RandomForestClassifier(criterion='gini',
                                 max_depth=None,
                                 min_samples_leaf=0.001,
                                 min_samples_split=0.001,
                                 n_estimators=100)
rf_gs_model.fit(X_train, y_train)

predictions = rf_gs_model.predict(X_test)
actuals = y_test

In [44]:
# new model report
model_report(rf_gs_model)

train score: 85.95%
test score: 75.0%
cross val score: 75.58%
AUC  : 54.65%

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.98      0.85       370
           1       0.68      0.11      0.19       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.72      0.55      0.52       504
weighted avg       0.73      0.75      0.68       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,363,7,370
1,119,15,134
All,482,22,504


## adaboost

In [45]:
#initialize and fit model
ada_model = AdaBoostClassifier()
ada_model.fit(X_train, y_train)

predictions = ada_model.predict(X_test)
actuals = y_test

In [46]:
#baseline model report
model_report(ada_model)

train score: 77.41%
test score: 74.4%
cross val score: 74.07%
AUC  : 58.53%

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.92      0.84       370
           1       0.54      0.25      0.34       134

   micro avg       0.74      0.74      0.74       504
   macro avg       0.66      0.59      0.59       504
weighted avg       0.71      0.74      0.71       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,342,28,370
1,101,33,134
All,443,61,504


In [47]:
param_grid_ada = [
    {'ada_model__n_estimators': [180,200,230],
    'ada_model__learning_rate': [0.25,0.30,.35]}
]

# Construct Grid Search
gs_ada = GridSearchCV(estimator=pipe_ada,
                    param_grid=param_grid_ada,
                    scoring='accuracy',
                    cv=4, n_jobs=-1, verbose=1)

# Fit using grid search
gs_ada.fit(X_train, y_train)

# Best accuracy and parameters
print('best accuracy: {:.4}%'.format(gs_ada.best_score_ * 100))
print('params:\n', gs_ada.best_params_)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   33.5s finished


best accuracy: 75.57%
params:
 {'ada_model__learning_rate': 0.3, 'ada_model__n_estimators': 200}


In [48]:
ada_gs_model = AdaBoostClassifier(n_estimators=200,
                                   learning_rate=0.3)
ada_gs_model.fit(X_train, y_train)

predictions = ada_gs_model.predict(X_test)
actuals = y_test


In [49]:
model_report(ada_gs_model)

train score: 78.25%
test score: 74.8%
cross val score: 74.78%
AUC  : 57.61%

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.94      0.85       370
           1       0.57      0.21      0.31       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.58      0.58       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,349,21,370
1,106,28,134
All,455,49,504


## gradient boost

In [50]:
#initialize and fit gradient boost
gbt_model = GradientBoostingClassifier()
gbt_model.fit(X_train, y_train)

predictions = gbt_model.predict(X_test)
actuals = y_test

In [51]:
#baseline model report
model_report(gbt_model)

train score: 82.87%
test score: 75.4%
cross val score: 74.94%
AUC  : 58.02%

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.95      0.85       370
           1       0.61      0.21      0.31       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.69      0.58      0.58       504
weighted avg       0.73      0.75      0.71       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,106,28,134
All,458,46,504


In [52]:
# grid search params
param_grid_gbt = [
    {'gbt_model__n_estimators': [150,200],
    'gbt_model__learning_rate': [0.1,0.01],
    'gbt_model__min_samples_split': [0.1, 0.01],
    'gbt_model__min_samples_leaf': [0.1, 0.01],
    'gbt_model__max_depth': [10, 15]}
]

# grid search
gs_gbt = GridSearchCV(estimator=pipe_gbt,
                    param_grid=param_grid_gbt,
                    scoring='accuracy',
                    cv=4, n_jobs=-1, verbose=1)

# fit grid search
gs_gbt.fit(X_train, y_train)

# best params
print('best accuracy: {:.4}%'.format(gs_gbt.best_score_ * 100))
print('params:\n', gs_gbt.best_params_)

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:  5.4min finished


best accuracy: 75.82%
params:
 {'gbt_model__learning_rate': 0.01, 'gbt_model__max_depth': 15, 'gbt_model__min_samples_leaf': 0.01, 'gbt_model__min_samples_split': 0.1, 'gbt_model__n_estimators': 200}


In [53]:
gbt_gs_model = GradientBoostingClassifier(learning_rate=0.01,
                                         max_depth=15,
                                         min_samples_leaf=0.01,
                                         min_samples_split=0.1,
                                         n_estimators=200)
gbt_gs_model.fit(X_train, y_train)

predictions = gbt_gs_model.predict(X_test)
actuals = y_test

In [54]:
# new model report
model_report(gbt_gs_model)

train score: 82.52%
test score: 74.21%
cross val score: 74.66%
AUC  : 56.01%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.84       370
           1       0.55      0.17      0.26       134

   micro avg       0.74      0.74      0.74       504
   macro avg       0.65      0.56      0.55       504
weighted avg       0.70      0.74      0.69       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,351,19,370
1,111,23,134
All,462,42,504


## XGB

In [55]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

predictions = xgb_model.predict(X_test)
actuals = y_test

In [56]:
model_report(xgb_model)

train score: 81.13%
test score: 74.8%
cross val score: 75.14%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [57]:
param_grid_xgb = [
    {'xgb_model__min_child_weight': [0.01,0.05,0.001],
    'xgb_model__learning_rate': [0.05,0.1],
    'xgb_model__n_estimators': [40,50],
    'xgb_model__max_depth': [3,4]}
]

# Construct Grid Search
gs_xgb = GridSearchCV(estimator=pipe_xgb,
                    param_grid=param_grid_xgb,
                    scoring='accuracy',
                    cv=4, n_jobs=-1, verbose=1)

# Fit using grid search
gs_xgb.fit(X, y)

# Best accuracy and parameters
print('Best accuracy: {:.4}%'.format(gs_xgb.best_score_ * 100))
print('params:\n', gs_xgb.best_params_)

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   43.1s finished


Best accuracy: 75.97%
params:
 {'xgb_model__learning_rate': 0.05, 'xgb_model__max_depth': 3, 'xgb_model__min_child_weight': 0.01, 'xgb_model__n_estimators': 50}


In [58]:
xgb_gs_model = XGBClassifier(learning_rate=0.05,
                             max_depth=3,
                             min_child_weight=0.01,
                             n_estimators=50)
xgb_gs_model.fit(X_train, y_train)

predictions = xgb_gs_model.predict(X_test)
actuals = y_test

In [59]:
model_report(xgb_gs_model)

train score: 81.58%
test score: 75.0%
cross val score: 75.74%
AUC  : 57.27%

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.95      0.85       370
           1       0.59      0.19      0.29       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.68      0.57      0.57       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,108,26,134
All,460,44,504


In [60]:
# xgb_gs_model_train_score = xgb_gs_model.score(X_train, y_train)
# print("train score: {:.4}%".format(xgb_gs_model_train_score * 100))

# xgb_gs_model_test_score = xgb_gs_model.score(X_test, y_test)
# print("test score: {:.4}%".format(xgb_gs_model_test_score * 100))

# xgb_gs_cv_score = np.mean(cross_val_score(gbt_gs_model, X, y, cv=3))
# print("cross val score: {:.4}%".format(xgb_gs_cv_score * 100))

# false_positive_rate, true_positive_rate, thresholds = roc_curve(actuals, predictions)
# roc_auc = auc(false_positive_rate, true_positive_rate)
# print("AUC : {:.4}%".format(roc_auc * 100))

# print("\nClassification Report:")
# print(classification_report(actuals, predictions))

# print("\nConfusion Matrix: 1=Fatal, 0=Minor")
# pd.crosstab(actuals, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)

In [61]:
# model = XGBClassifier(random_state=3)
# model.fit(X_train, y_train)
# train_preds = model.predict(X_train)
# val_preds = model.predict(X_test)
# training_accuracy = accuracy_score(y_train, train_preds)
# val_accuracy = accuracy_score(y_test, val_preds)
# training_precision = precision_score(y_train, train_preds)
# val_precision = precision_score(y_test, val_preds)

# print("Training Accuracy:   {:.4}%".format(training_accuracy * 100))
# print("Validation Accuracy: {:.4}%".format(val_accuracy * 100))
# print('----------------------------')
# print("Training precision:   {:.4}%".format(training_precision * 100))
# print("Validation precision: {:.4}%".format(val_precision * 100))
# print('----------------------------')
# print("Training Confusion Matrix:")
# print(confusion_matrix(y_train, train_preds))
# print("Validation Confusion Matrix:")
# print(confusion_matrix(y_test, val_preds))

In [62]:
# param_grid_xgb = [
#     {'xgb_model__min_child_weight': [0.0001, 0.001],
#     'xgb_model__learning_rate': [0.1],
#     'xgb_model__n_estimators': [50],
#     'xgb_model__max_depth': [2, 3]}
# ]

# # Construct Grid Search
# gs_xgb = GridSearchCV(estimator=pipe_xgb,
#                     param_grid=param_grid_xgb,
#                     scoring='accuracy',
#                     cv=4, n_jobs=-1, verbose=1)

# # Fit using grid search
# gs_xgb.fit(X_train, y_train)

# # Best accuracy and parameters
# print('Best accuracy: {:.4}%'.format(gs_xgb.best_score_ * 100))
# print('Best Params:\n', gs_xgb.best_params_)

In [63]:
# xgb_gs_model = XGBClassifier(learning_rate=0.1,
#                              max_depth=3,
#                              min_child_weight=0.001,
#                              n_estimators=50)
# xgb_gs_model.fit(X_train, y_train)

# predictions = xgb_gs_model.predict(X_test)
# actuals = y_test

In [64]:
# xgb_gs_model_train_score = xgb_gs_model.score(X_train, y_train)
# print("Training Accuracy: {:.4}%".format(xgb_gs_model_train_score * 100))

# xgb_gs_model_test_score = xgb_gs_model.score(X_test, y_test)
# print("Testing Accuracy: {:.4}%".format(xgb_gs_model_test_score * 100))

# xgb_gs_cv_score = np.mean(cross_val_score(gbt_gs_model, X_train, y_train, cv=3))
# print("Cross Val Score: {:.4}%".format(xgb_gs_cv_score * 100))

# false_positive_rate, true_positive_rate, thresholds = roc_curve(actuals, predictions)
# roc_auc = auc(false_positive_rate, true_positive_rate)
# print("AUC is : {:.4}%".format(roc_auc * 100))

# print("\nClassification Report:")
# print(classification_report(actuals, predictions))

# print("\nConfusion Matrix: 1=Fatal, 0=Minor")
# pd.crosstab(actuals, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)

## SVC

In [82]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

predictions = gbt_model.predict(X_test)
actuals = y_test

In [83]:
model_report(svm_model)

train score: 83.12%
test score: 74.21%
cross val score: 73.27%
AUC  : 58.02%

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.95      0.85       370
           1       0.61      0.21      0.31       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.69      0.58      0.58       504
weighted avg       0.73      0.75      0.71       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,106,28,134
All,458,46,504


In [84]:
# Construct pipeline for support vector grid search

pipe_svm = Pipeline([('pca', PCA(n_components=115)),
            ('clf', svm.SVC(random_state=3))])

# Set grid search params
param_grid_svm = [
 # {'clf__C': [0.1, 1]  , 'clf__kernel': ['linear']},
  {'clf__C': [0.1, 1, 10], 'clf__gamma': [0.001, 0.01, 0.1], 'clf__kernel': ['rbf','linear']},
 ]

# Construct grid search
gs_svm = GridSearchCV(estimator=pipe_svm,
            param_grid=param_grid_svm,
            scoring='accuracy',
            cv=3, verbose=2, return_train_score = True)

# Fit using grid search
gs_svm.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_svm.best_score_)

# Best params
print('\nBest params:\n', gs_svm.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] clf__C=0.1, clf__gamma=0.001, clf__kernel=rbf ...................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... clf__C=0.1, clf__gamma=0.001, clf__kernel=rbf, total=   0.4s
[CV] clf__C=0.1, clf__gamma=0.001, clf__kernel=rbf ...................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] .... clf__C=0.1, clf__gamma=0.001, clf__kernel=rbf, total=   0.4s
[CV] clf__C=0.1, clf__gamma=0.001, clf__kernel=rbf ...................
[CV] .... clf__C=0.1, clf__gamma=0.001, clf__kernel=rbf, total=   0.4s
[CV] clf__C=0.1, clf__gamma=0.001, clf__kernel=linear ................
[CV] . clf__C=0.1, clf__gamma=0.001, clf__kernel=linear, total=   0.5s
[CV] clf__C=0.1, clf__gamma=0.001, clf__kernel=linear ................
[CV] . clf__C=0.1, clf__gamma=0.001, clf__kernel=linear, total=   0.5s
[CV] clf__C=0.1, clf__gamma=0.001, clf__kernel=linear ................
[CV] . clf__C=0.1, clf__gamma=0.001, clf__kernel=linear, total=   0.7s
[CV] clf__C=0.1, clf__gamma=0.01, clf__kernel=rbf ....................
[CV] ..... clf__C=0.1, clf__gamma=0.01, clf__kernel=rbf, total=   0.5s
[CV] clf__C=0.1, clf__gamma=0.01, clf__kernel=rbf ....................
[CV] ..... clf__C=0.1, clf__gamma=0.01, clf__kernel=rbf, total=   0.5s
[CV] clf__C=0.1, clf__gamma=0.01, clf__kernel=rbf ....................
[CV] .

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  4.3min finished


Best accuracy: 0.750

Best params:
 {'clf__C': 0.1, 'clf__gamma': 0.001, 'clf__kernel': 'rbf'}


In [85]:
svm_gs_model = XGBClassifier(clf_C=0.1,
                             clf_gamma=0.001,
                             clf_kernel='rbf',
                             )
svm_gs_model.fit(X_train, y_train)

predictions = svm_gs_model.predict(X_test)
actuals = y_test

In [86]:
model_report(svm_gs_model)

train score: 81.13%
test score: 74.8%
cross val score: 75.14%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [88]:
#from sklearn import svm, grid_search
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [89]:
svc_param_selection(X_train, y_train, 5)

{'C': 10, 'gamma': 0.001}

In [81]:
model_report(svm_gs_model)

train score: 81.13%
test score: 74.8%
cross val score: 75.14%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [66]:
# grid search
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100]},
                    {'kernel': ['linear'], 'C': [1, 10]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        svm.SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.375 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.375 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.635 (+/-0.045) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.458 (+/-0.236) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.561 (+/-0.029) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.635 (+/-0.020) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.613 (+/-0.022) for {'C': 1, 'kernel': 'linear'}
0.614 (+/-0.020) for {'C': 10, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.76      0.96      0.85       370
           1       0.57      0.15      0.24       134

   micro avg       0.74      0.74      0.74   

In [90]:
svm_gs_model = XGBClassifier(clf_C=10,
                             clf_gamma=0.001,
                             clf_kernel='rbf',
                             )
svm_gs_model.fit(X_train, y_train)

predictions = svm_gs_model.predict(X_test)
actuals = y_test

In [91]:
model_report(svm_gs_model)

train score: 81.13%
test score: 74.8%
cross val score: 75.14%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


## model reports

In [95]:
model_report(dt_gs_model)

train score: 77.06%
test score: 73.21%
cross val score: 75.14%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [96]:
model_report(rf_gs_model)

train score: 85.95%
test score: 75.0%
cross val score: 75.46%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [97]:
model_report(ada_gs_model)

train score: 78.25%
test score: 74.8%
cross val score: 74.78%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [98]:
model_report(xgb_gs_model)

train score: 81.58%
test score: 75.0%
cross val score: 75.74%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [99]:
model_report(svm_gs_model)

train score: 81.13%
test score: 74.8%
cross val score: 75.14%
AUC  : 56.9%

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.95      0.85       370
           1       0.58      0.19      0.28       134

   micro avg       0.75      0.75      0.75       504
   macro avg       0.67      0.57      0.56       504
weighted avg       0.72      0.75      0.70       504


Confusion Matrix: 1=Fatal, 0=Minor


Predicted,0,1,All
Actual,,,
0,352,18,370
1,109,25,134
All,461,43,504


In [ ]:
svc_param_selection(X_train, y_train, 5)

In [67]:
# pipe_kn = Pipeline([('scale', StandardScaler()),
#                     ('kn_model', KNeighborsClassifier())])

# pipe_tree = Pipeline([('dt_model', DecisionTreeClassifier(random_state=81))])

# pipe_bag = Pipeline([('bag_model', BaggingClassifier(DecisionTreeClassifier(random_state=81)))])

# pipe_rf = Pipeline([('rf_model', RandomForestClassifier(random_state=81))])

# pipe_ada = Pipeline([('ada_model', AdaBoostClassifier(random_state=81))])

# pipe_gbt = Pipeline([('gbt_model', GradientBoostingClassifier(random_state=81))])

# pipe_xgb = Pipeline([('xgb_model', XGBClassifier(random_state=81))])

In [69]:
# def plot_feature_importances(model):
#     n_features = X.shape[1]
#     plt.figure(figsize=(8,8))
#     plt.barh(range(n_features), model.feature_importances_, align='center') 
#     plt.yticks(np.arange(n_features), X.columns.values) 
#     plt.xlabel("Feature importance")
#     plt.ylabel("Feature")

In [70]:
# for index, val in enumerate(pipelines):
#     print('%s pipeline test accuracy: %.3f' % (pipeline_names[index], val.score(X_test, y_test)))

KNeighbors pipeline test accuracy: 0.690
Decision Tree pipeline test accuracy: 0.661
Bagged Trees pipeline test accuracy: 0.752
Random Forest pipeline test accuracy: 0.742
AdaBoost pipeline test accuracy: 0.744
Gradient Boosting pipeline test accuracy: 0.754
XGBoost pipeline test accuracy: 0.748


Decision tree

In [73]:
#gs_rf.cv_results_

In [74]:
#gs_ab.cv_results_

In [75]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [76]:
#gs_svm.cv_results_

## IGNORE BELOW

In [77]:
# from sklearn.linear_model import SGDClassifier
# sgd_clf = SGDClassifier(random_state=123)
# sgd_clf.fit(X_train, y_train_fatal)

In [78]:
# from sklearn.model_selection import cross_val_score
# cross_val_score(sgd_clf, X_train, y_train_fatal, cv=3, scoring="accuracy")

In [79]:
# from sklearn.metrics import precision_score, recall_score, f1_score

print("accuracy score:")
precision_score(y_train_fatal, y_train_pred)

print ("recall score:")
recall_score(y_train_fatal, y_train_pred)

print("f1 score:")
f1_score(y_train_fatal, y_train_pred)

accuracy score:


NameError: name 'y_train_fatal' is not defined

In [ ]:
# y_scores = cross_val_predict(sgd_clf, X_train, y_train_fatal, cv=3, method="decision_function")

In [ ]:
# from sklearn.metrics import precision_recall_curve
# precisions, recalls, thresholds = precision_recall_curve(y_train_fatal, y_scores)

In [ ]:
# def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
#     plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
#     plt.plot(thresholds, recalls[:-1], "g--", label="Recall")

In [ ]:
# plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
# plt.show()

In [ ]:
# from sklearn.metrics import roc_curve
# fpr, tpr, thresholds = roc_curve(y_train_fatal, y_scores)

In [ ]:
# def plot_roc_curve(fpr, tpr, label=None):
#     plt.plot(fpr, tpr, linewidth=2, label=label)
#     plt.plot([0,1], [0,1], 'k--')

In [ ]:
# plot_roc_curve(fpr, tpr)

In [ ]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_train_fatal, y_scores)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# forest_clf = RandomForestClassifier(random_state=42)
# y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_fatal, cv=3, method="predict_proba")

In [ ]:
# y_scores_forest = y_probas_forest[:, 1]
# fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_fatal, y_scores_forest)

In [ ]:
# plt.plot(fpr, tpr, "b:", label='SGD')
# plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
# roc_auc_score(y_train_fatal, y_scores_forest)

In [ ]:
# # Model Evaluation metrics 
# from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
# print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
# # print('Precision Score : ' + str(precision_score(y_test,y_pred)))
# # print('Recall Score : ' + str(recall_score(y_test,y_pred)))
# # print('F1 Score : ' + str(f1_score(y_test,y_pred)))

# #Dummy Classifier Confusion matrix
# from sklearn.metrics import confusion_matrix
# print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

In [ ]:
# from sklearn.svm import SVC
# svm_clf = SVC()
# svm_clf.fit(X_train[:300], y_train[:300])
# #svm_clf.predict(X[0])

In [ ]:
# cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring = "accuracy")

In [93]:
random_state=3

In [94]:
# Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


clf_A = RandomForestClassifier(random_state=random_state)
clf_B = GaussianNB()
clf_C = SVC(random_state=random_state)

# Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_100 = len(y_train)
samples_10 = int(len(y_train)/10)
samples_1 = int(len(y_train)/100)

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10,   samples_100]):
        results[clf_name][i] = \
        y_train_pred(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
vs.evaluate(results, accuracy, fscore)

NameError: name 'y_train_pred' is not defined